This notebook contains a demonstration of the fulle procedure to use a grasping service to execute a grap on a loaded ycb object. 
The procedure is the following:

1.) load an yccb object

2.) put the object into the simulated scene and take a picture from a defined viewpoint

3.) create a grasp planner request from the recorded data

4.) send the request to the ROS grasping service

5.) wait for the result and decompose it

6.) execute a grasping sequence based on the decomposed result

To successfully execute this notebook, the grasping service needs to be running.
The grasping services from the `grasping-benchmark-panda` repo can be easily started using the Docker containers provided in the `grasping-benchmark-panda` repo. 
Simply run `docker-compose -f <path-to-grasping-benchmark-panda/docker/build/docker-compose.yaml> up <name-of-the-grasp-algo>`. 

In [1]:
from pathlib import Path
import logging
import importlib
import sys

from scipy.spatial.transform import Rotation as R
import numpy as np

from alr_sim.core import Scene

from alr_sim_tools.ycb_utils import YCBLoader
from alr_sim_tools.scene_utils import (
    execute_grasping_sequence,
    record_camera_data,
    reset_scene,
)
from alr_sim_tools.ros_utils import create_grasp_planner_request

sys.path.append(str(Path.cwd().parent / "alr_sim_tools" / "ros_msg_srv_definitions"))
from alr_sim_tools.ros_msg_srv_definitions.grasping_benchmarks_ros.srv import (
    GraspPlannerResponse,
    GraspPlanner,
)

import rospy

pybullet build time: May  2 2023 05:55:04


In [2]:
FACTORY_STRING = "mj_beta"
OBJECT_POS = (0.5, 0.0, 0.2)
CAM_POS = (0.5, 0.0, 1)
CAM_QUAT = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
CAM_HEIGHT = 480
CAM_WIDTH = 640
INITIAL_ROBOT_POS = (0.0, 0.5, 0.2)
INITIAL_ROBOT_QUAT = (0, 1, 0, 0)
RENDER_MODE = Scene.RenderMode.HUMAN    

GRASP_ALGO_NAME = "grconvnet"
GRASP_PLANNER_SERVICE_ID = f"{GRASP_ALGO_NAME}_bench/{GRASP_ALGO_NAME}_grasp_planner_service"

YCB_FOLDER = Path.home() / "Documents" / "SF-ObjectDataset" / "YCB"
YCB_OBJECT_ID = "011_banana"

In [3]:
ycb_loader = YCBLoader(ycb_base_folder=YCB_FOLDER, factory_string="mj_beta")

sim_obj = ycb_loader.get_ycb_object(
    pos=OBJECT_POS, quat=(0, 1, 0, 0), obj_id=YCB_OBJECT_ID, name=YCB_OBJECT_ID, grounded=True
)

In [4]:
camera_data, scene, agent = record_camera_data(
    factory_string=FACTORY_STRING,
    cam_pos=CAM_POS,
    cam_quat=CAM_QUAT,
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    robot_pos=INITIAL_ROBOT_POS,
    robot_quat=INITIAL_ROBOT_QUAT,
    object_list=[sim_obj],
    target_obj_name=YCB_OBJECT_ID,
    render_mode=RENDER_MODE,
    wait_time=1,
)

In [5]:
rospy.init_node("grasp_planner_client")
importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

In [6]:
grasp_req = create_grasp_planner_request(
    rgb_img=camera_data.rgb_img,
    depth_img=camera_data.depth_img,
    seg_img=camera_data.seg_img,
    pc_points=camera_data.point_cloud_points,
    pc_colors=camera_data.point_cloud_colors,
    cam_pos=camera_data.cam_pos,
    cam_quat=camera_data.cam_quat,
    cam_intrinsics=camera_data.cam_intrinsics,
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    num_of_candidates=1,
)

In [7]:
rospy.wait_for_service(GRASP_PLANNER_SERVICE_ID, timeout=30.0)
grasp_planner = rospy.ServiceProxy(GRASP_PLANNER_SERVICE_ID, GraspPlanner)

reply: GraspPlannerResponse = grasp_planner(grasp_req)

In [8]:
best_grasp = sorted(reply.grasp_candidates, key=lambda x: x.score.data)[-1]

grasp_pos = best_grasp.pose.pose.position
grasp_pos = np.array((grasp_pos.x, grasp_pos.y, grasp_pos.z))

grasp_quat = best_grasp.pose.pose.orientation
grasp_quat = np.array((grasp_quat.w, grasp_quat.x, grasp_quat.y, grasp_quat.z))

grasp_rot = R.from_quat(grasp_quat[[1,2,3,0]]).as_matrix()

In [9]:
execute_grasping_sequence(
    agent = agent,
    grasp_pos = grasp_pos,
    grasp_quat = grasp_quat
)

INFO:root:Beam to hover_ position [0.6884302  0.26474797 0.051     ]
/home/moritz/miniconda3/envs/alr_tools/lib/python3.9/site-packages/glfw/__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
INFO:root:Opening gripper
INFO:root:Going to grasp position [0.6884302  0.26474797 0.001     ]


KeyboardInterrupt: 

In [ ]:
reset_scene(FACTORY_STRING, scene, agent)